In [ ]:
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine

aujourdhui = datetime.now()
date_controle = aujourdhui.strftime('%d-%m-%y')

user = 'airflow'
password = 'airflow'
host = 'postgres'
port = '5432'  
database_bronze = 'bronze'
database_silver = 'silver'

db_url_bronze = f"postgresql://{user}:{password}@{host}:{port}/{database_bronze}"
engine_bronze = create_engine(db_url_bronze)

db_url_silver = f"postgresql://{user}:{password}@{host}:{port}/{database_silver}"
engine_silver= create_engine(db_url_silver)

df_clients = pd.read_sql("SELECT * FROM clients", con=engine_bronze)
df_controles = pd.read_sql("SELECT * FROM controles", con=engine_silver)


In [17]:
df_clients.head()

,ID_CLIENT,NOM,PRENOM,CIN,CARTE_SEJOUR,VILLE,NATIONALITE,TELEPHONE,id_intermediaire,DATE_CREATION
0,lu1cldwMLx,Hicks,Dawn,SJ109379,OL118027,None,mre,800-778-8075x457,84,2021-05-14
1,k8N4sn4YJb,Wolfe,Kimberly,B136474,O499923,North Alishaport,mre,+1-913-505-2118x425,15,2024-03-27
2,6qzPbzht3H,Burch,Isabel3@,KX010301,HM264290,North Eric`,mre,935-921-5809,13,2021-07-16
3,KZPbHdxc1a,None,Matthew,YT977001,OI901480,South Kenneth,etranger,4332103507,2,2023-02-25
4,3KaFE6Zhq9,Roth=#,None,EB353024,B240907,None,marocain,001-367-758-8791,91,2021-01-03


In [18]:
df_controles.head()

,ID_CONTROLE,DATA,DIM_CONTROLE,DATE_CONTROLE
0,1.0,NOM,COMPLETUDE,23-10-24
1,2.0,PRENOM,COMPLETUDE,23-10-24
2,3.0,CIN,COMPLETUDE,23-10-24
3,4.0,CARTE_SEJOUR,COMPLETUDE,23-10-24
4,5.0,VILLE,COMPLETUDE,23-10-24


### table resultat validite de la colonne nom

In [19]:
import pandas as pd
import re

def valider_chaine(chaine):
    if isinstance(chaine, str) and chaine:
        if not re.search(r'\s', chaine) and not re.search(r'\d', chaine) and re.search(r'^[A-Za-z]+$', chaine):
            return 1
    return 0


In [ ]:
df_val_nom = df_clients[['id_client']].copy()
df_val_nom['result'] = df_clients['nom'].apply(valider_chaine) 

max_id = df_controles['id_controle'].max()
df_val_nom['id_controle'] = max_id + 1 


In [ ]:
df_val_nom.to_sql('df_val_nom', con=engine_silver, if_exists='replace', index=False)

### remplissage de la table des controles

In [ ]:
id_controle = df_controles['id_controle'].max() + 1
data = "nom"
dim_controle = "validite"

nouvelles_lignes = [[id_controle, data, dim_controle, date_controle]]
nouvelle_ligne_df = pd.DataFrame(nouvelles_lignes, columns=['id_controle', 'data', 'dim_controle', 'date_controle'])
df_controles = pd.concat([df_controles, nouvelle_ligne_df], ignore_index=True)

### table resultat validite de la colonne prenom

In [ ]:
df_val_prenom = df_clients[['id_client']].copy()
df_val_prenom['result'] = df_clients['prenom'].apply(valider_chaine) 

max_id = df_controles['id_controle'].max()
df_val_prenom['id_controle'] = max_id + 1 


In [ ]:
df_val_prenom.to_sql('df_val_prenom', con=engine_silver, if_exists='replace', index=False)

### remplissage de la table des controles

In [ ]:
id_controle = df_controles['id_controle'].max() + 1
data = "prenom"
dim_controle = "validite"

nouvelles_lignes = [[id_controle, data, dim_controle, date_controle]]
nouvelle_ligne_df = pd.DataFrame(nouvelles_lignes, columns=['id_controle', 'data', 'dim_controle', 'date_controle'])
df_controles = pd.concat([df_controles, nouvelle_ligne_df], ignore_index=True)

### table resultat validite de la colonne cin

In [ ]:
import re
import pandas as pd
import numpy as np  

def valider_cin(cin):
    
    if pd.isnull(cin):
        return 0 
    
    
    if isinstance(cin, str) and re.match(r'^[A-Za-z]{1,2}\d{6}$', cin):
        return 1 
    return 0 


In [ ]:
df_val_cin = df_clients[['id_client']].copy()
df_val_cin['result'] = df_clients['cin'].apply(valider_cin) 

max_id = df_controles['id_controle'].max()
df_val_cin['id_controle'] = max_id + 1 


In [ ]:
df_val_cin.to_sql('df_val_cin', con=engine_silver, if_exists='replace', index=False)

### remplissage de la table des controles

In [ ]:
id_controle = df_controles['id_controle'].max() + 1
data = "cin"
dim_controle = "validite"

nouvelles_lignes = [[id_controle, data, dim_controle, date_controle]]
nouvelle_ligne_df = pd.DataFrame(nouvelles_lignes, columns=['id_controle', 'data', 'dim_controle', 'date_controle'])
df_controles = pd.concat([df_controles, nouvelle_ligne_df], ignore_index=True)

### table resultat validite de la colonne carte sejour

In [ ]:
import re
import pandas as pd
import numpy as np  

def valider_carte_sejour(carte):
   
    if pd.isnull(carte):
        return 0 
    
   
    if isinstance(carte, str) and re.match(r'^\d{1,10}$', carte):
        return 1 
    return 0  


In [ ]:
df_val_carte_sejour = df_clients[['id_client']].copy()
df_val_carte_sejour['result'] = df_clients['carte_sejour'].apply(valider_carte_sejour) 

max_id = df_controles['id_controle'].max()
df_val_carte_sejour['id_controle'] = max_id + 1 


In [ ]:
df_val_carte_sejour.to_sql('df_val_carte_sejour', con=engine_silver, if_exists='replace', index=False)

### remplissage de la table des controles

In [ ]:
id_controle = df_controles['id_controle'].max() + 1
data = "carte_sejour"
dim_controle = "validite"

nouvelles_lignes = [[id_controle, data, dim_controle, date_controle]]
nouvelle_ligne_df = pd.DataFrame(nouvelles_lignes, columns=['id_controle', 'data', 'dim_controle', 'date_controle'])
df_controles = pd.concat([df_controles, nouvelle_ligne_df], ignore_index=True)

### table resultat validite de la colonne ville

In [ ]:
import re
import pandas as pd
import numpy as np  
def valider_ville(ville):

    if pd.isnull(ville):
        return 0  
    

    ville = ville.strip().lower()
    
   
    if re.match(r'^[a-z]+( [a-z]+)*$', ville):
        return 1 
    return 0  


In [ ]:
df_val_ville = df_clients[['id_client']].copy()
df_val_ville['result'] = df_clients['ville'].apply(valider_ville) 

max_id = df_controles['id_controle'].max()
df_val_ville['id_controle'] = max_id + 1 


In [ ]:
df_val_ville.to_sql('df_val_ville', con=engine_silver, if_exists='replace', index=False)

### remplissage de la table des controles

In [ ]:
id_controle = df_controles['id_controle'].max() + 1
data = "ville"
dim_controle = "validite"

nouvelles_lignes = [[id_controle, data, dim_controle, date_controle]]
nouvelle_ligne_df = pd.DataFrame(nouvelles_lignes, columns=['id_controle', 'data', 'dim_controle', 'date_controle'])
df_controles = pd.concat([df_controles, nouvelle_ligne_df], ignore_index=True)

### table resultat validite de la colonne telephone

In [ ]:
import re
import pandas as pd

def valider_telephone(numero):

    if pd.isnull(numero):
        return 0  
    
 
    if re.match(r'^\+?[0-9()]*$', numero):
        return 1  
    return 0  


In [ ]:
df_val_telephone = df_clients[['id_client']].copy()
df_val_telephone['result'] = df_clients['telephone'].apply(valider_telephone) 

max_id = df_controles['id_controle'].max()
df_val_telephone['id_controle'] = max_id + 1 


In [ ]:
df_val_telephone.to_sql('df_val_telephone', con=engine_silver, if_exists='replace', index=False)

### remplissage de la table des controles

In [ ]:
id_controle = df_controles['id_controle'].max() + 1
data = "telephone"
dim_controle = "validite"

nouvelles_lignes = [[id_controle, data, dim_controle, date_controle]]
nouvelle_ligne_df = pd.DataFrame(nouvelles_lignes, columns=['id_controle', 'data', 'dim_controle', 'date_controle'])
df_controles = pd.concat([df_controles, nouvelle_ligne_df], ignore_index=True)

In [ ]:
df_controles.to_sql('controles', con=engine_silver, if_exists='append', index=False)